In [1]:
class Room:
    def __init__(self, name, description):
        self.name = name
        self.description = description
        self.connections = []
        self.items = []

class Item:
    def __init__(self, name, description):
        self.name = name
        self.description = description

class Player:
    def __init__(self):
        self.current_room = None
        self.inventory = []

    #def move(self, new_room):
        # Logic for moving between rooms

